# Gettting Started with the Icestick

Adapted from
[https://github.com/rdaly525/CS448H/blob/master/icestick/README.md](https://github.com/rdaly525/CS448H/blob/master/icestick/README.md)
by Ross Daly

### Lattice Icestick
[http://www.latticesemi.com/icestick](http://www.latticesemi.com/icestick)

> USB thumb drive form factor evaluation board - [...] an easy to use, small size board that allows rapid prototyping of system functions at a very low cost using Lattice Semiconductor's iCE40 FPGA family.

### Project Icestorm
[http://www.clifford.at/icestorm/](http://www.clifford.at/icestorm/)

> Project IceStorm aims at reverse engineering and documenting the bitstream
> format of Lattice iCE40 FPGAs and providing simple tools for analyzing and
> creating bitstream files. The IceStorm flow (Yosys, Arachne-pnr, and
> IceStorm) is a fully open source Verilog-to-Bitstream flow for iCE40 FPGAs.

## Setup
Follow the installation instructions on the [the icestorm website]([http://www.clifford.at/icestorm/](http://www.clifford.at/icestorm/) under "Where are the Tools? How to install?"

In [1]:
with open("blink.py", "r") as blink:
    print(blink.read())

from magma import wire, compile, EndCircuit
from loam.boards.icestick import IceStick, Counter

icestick = IceStick()
icestick.Clock.on()
icestick.D1.on()

main = icestick.main()

counter = Counter(24)
wire(counter.O[-1], main.D1)

EndCircuit()



We can use the `magma` binary to compile `blink.py` for the icestick.

In [2]:
!../../bin/magma -b icestick blink.py

import mantle lattice ice40
import mantle lattice mantle40
compiling FullAdder
compiling AdderCout24
compiling Register24
compiling Counter24
compiling main


To inspect the generated verilog

In [3]:
with open("build/blink.v", "r") as blink_verilog:
    print(blink_verilog.read())

module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module AdderCout24 (input [23:0] I0, input [23:0] I1, output [23:0] O, output  COUT);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
wire  inst4_O;
wire  inst4_COUT;
wire  inst5_O;
wire  inst5_COUT;
wire  inst6_O;
wire  inst6_COUT;
wire  inst7_O;
wire  inst7_COUT;
wire  inst8_O;
wire  inst8_COUT;
wire  inst9_O;
wire  inst9_COUT;
wire  inst10_O;
wire  inst10_COUT;
wire  inst11_O;
wire  inst11_COUT;
wire  inst12_O;
wire  inst12_COUT;
wire  inst13_O;
wire  inst13_COUT;
wire  inst14_O;
wire  inst14_COUT;
wire  inst15_O;
wire  inst15_COUT;
wire  inst16_O;
wire  inst16_COUT;
wire  inst17_O;
wire  in

Also the generated pcf

In [4]:
with open("build/blink.pcf", "r") as blink_pcf:
    print(blink_pcf.read())

set_io D1 99
set_io CLKIN 21



To flash the `blink` circuit onto the icestick using `yosys`, `arachne-pnr` and the `icestorm` tools

In [5]:
%%bash
yosys -q -p 'synth_ice40 -top main -blif build/blink.blif' build/blink.v
arachne-pnr -q -d 1k -o build/blink.txt -p build/blink.pcf build/blink.blif 
icepack build/blink.txt build/blink.bin
iceprog build/blink.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


To view the timing analysis

In [6]:
!icetime -tmd hx1k build/blink.txt

// Reading input .asc file..
// Reading 1k chipdb file..
// Creating timing netlist..

icetime topological timing analysis report

Info: max_span_hack is enabled: estimate is conservative.

Report for critical path:
-------------------------

        lc40_6_5_3 (LogicCell40) [clk] -> lcout: 0.640 ns
     0.640 ns net_10762 (inst0.inst0.inst3.I0)
        odrv_6_5_10762_539 (Odrv12) I -> O: 0.540 ns
        t37 (LocalMux) I -> O: 0.330 ns
        inmux_7_5_15027_15070 (InMux) I -> O: 0.260 ns
        lc40_7_5_3 (LogicCell40) in1 -> carryout: 0.260 ns
     2.029 ns net_15068 (inst0.inst0.inst3.inst1_CO$2)
        lc40_7_5_4 (LogicCell40) carryin -> carryout: 0.126 ns
     2.155 ns net_15074 (inst0.inst0.inst4.inst1_CO$2)
        lc40_7_5_5 (LogicCell40) carryin -> carryout: 0.126 ns
     2.281 ns net_15080 (inst0.inst0.inst5.inst1_CO$2)
        lc40_7_5_6 (LogicCell40) carryin -> carryout: 0.126 ns
     2.407 ns net_15086 (inst0.inst0.inst6.inst1_CO$2)
        lc40_7_5_7 (LogicCell40) car